# Efficient finetuning of Llama 3 with FSDP QDora on the Uganda Clinical Guidelines

## Background

The Ugandan Ministry of Health and its partners published the Uganda Clinical Guidelines to help give practitioners access to the latest up-to-date information on how to diagnose and manage common health conditions in Uganda.

You can find a link to the Uganda clinical guidelines [here](https://library.health.go.ug/uganda-clinical-guidelines-2023).

To quote the Clinical guidelines book itself,

> **What is the aim of the UCG?**
> 
>The UCG aims to provide summarized easy-to-use, practical, complete and useful information on how to quickly and correctly diagnose and manage common conditions you are likely to encounter. This will ensure that patients receive the best possible clinical services and obtain prompt and effective relief from or cure of their complaint, thereby making the most appropriate use of scarce diagnostic and clinical resources, including medicines. It should, however, be emphasised that the UCG does not replace or substituteavailable textbooks on the subject.

> **Why is the UCG necessary?**
>
> Medicine is an ever-evolving and expanding field in terms of needs and knowledge. The UCG helps the country to prioritize and effectively use limited resources by guiding the procurement system to ensure the availability of the most needed medicines and supplies. In the context of new knowledge and changing priorities, as a tool, the UCG assists health workers in their daily practice by providing information in an easy-to-follow and practical format.

With this, we are experimenting with fine-tuning a large language model like llama on these guidelines. The hope is that this model can be used as a basis for an assistive tool.

The Uganda clinical guidelines have over 1000+ pages containing information such as clinical features, causes, differential diagnoses, treatment, and prevention options for many common health complaints in Uganda.

## Training

I shall be fine-tuning a `Llama3`70B parameter model using FSDP QDora, an open-source system first introduced in this answer.ai [post](https://www.answer.ai/posts/2024-04-26-fsdp-qdora-llama3.html). This system is an extension of FSDP QLora which is a combination of FSDP and QLora.

### Background

QLora was made possible by 2 advances in neural networks, namely quantization and LORA. Quantization reduces the number of bits used to represent parameters in a model, here we find ourselves trading off between zero shot accuracy at inference time and model bits.  

For example instead of using 32 or 16 bits to store the weights of a neural network, we can use a smaller number of bits, like 4. A 4-bit number is equivalent to (2 x 2 x 2 x 2) and has only 16 possible values. 

*Tim Dettmers & Luke Zettlemoyer* released a paper that showed that they ran an experiments to determine the bit precision that maximizes one shot learning. We can see that the Zero-shot accuracy increases steadily for fixed model bits from 16 up to 4 bit quantitzation precision, When we reach 3 bits, we can see that the relationship reverses. Refer to the image below from the original [The case for 4-bit precision:
k-bit Inference Scaling Laws](https://arxiv.org/pdf/2212.09720) paper.






 4 bit precision was shown to be the best precision value that maximises total model bits and zero shot accuracy. zero shot accuracy and the Total model bits 


It is possible to create these 4-bit models using the **bitsandbytes** library by Tim Dettmers. The downside to this 

This method enables us to efficiently and cheaply fine-tune large models with billions of parameters on consumer hardware like standard gaming GPUS's, as opposed to using data center-class GPUs, which cost thousands of dollars more. Remember, each parameter usually takes 16 bits (2 bytes), so that’s 70*2=140GB to even store the weights without thinking of the other data like the gradients etc.(Full Sharded Data Parallel)[https://engineering.fb.com/2021/07/15/open-source/fsdp/] developed by Meta, which helps us distribute our parameters across multiple GPUs, allowing us to use them in parallel. This used in conjuction with QDora which brings quantitization to 

Quantization is a technique where instead of using 32 or 16 bits to store the weights of a neural network, 4 are used. It is possible to create these 4 bit models using the **bitsandbytes** library by Tim Dettmers. The downside to this 